<h1 align="center">Python Programming for Machine Learning</h1>
<h2 align="center">Acceleration Frameworks – Part 2</h2>

<center><img src='images/python-logo-only.svg' width=250> </center>

In [1]:
import numpy as np
import torch
import matplotlib

print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

NumPy version: 1.26.4
PyTorch version: 2.5.1
Matplotlib version: 3.9.2


Reminder:
- Lecturer: Have you started the recording?
- Audience: Have you received a notification that the recording has started?

# Tensor ops: bigger picture

This section:
- partially exam-relevant, e.g., the exam will **not** ask qualitative questions about JIT
- helps with coding tensor ops by understaning their context & helps with projects outside of PyML

## How to implement fast code "from first principles"?

### Bigger picture

Abstractions:
- Implementation: max runtime performance
- Interface: max development speed (readability, ecosystem)

Python advantages:
- Interface is clean **and** adaptable (e.g., libraries can overload operators like `+`): readable code **and** speed
- Mature ecosystem: user can benefit from interface without knowing C/CPP

### Tensor ops: abstraction layers

- GPU: machine code / assembly-like / PTX / ...
- Kernel/compiler/vendor libs: e.g., CUDA / CUBLAS / CUDNN / XLA / ...
- Python interface: torch / JAX / ...

## Understanding tensor ops without the "frontend"

Frontend: e.g., torch.matmul vs. einsum

> **🎯 Goal: attain general skills, don't overfit on torch syntax.**

[tinygrad](https://tinygrad.org/#tinygrad):
- Autograd enginge (similar to torch)
- Pro: can run on GPUs of many vendors
- Con: less mature, less adoption, less speed than specialized frameworks
- Method:
  - Compiler/abstraction over vendor-specific kernels
  - **Requires low complexity** (only three core ops)

> **💡 We can express all tensor ops using the core tinygrad ops (independent of frontend syntax)**

Practice: Express the below in "tinygrad ops":
- Broadcasting
- Matmul (note: [matmul intuition](https://pytorch.org/blog/inside-the-matrix/))

## Different "frontends"

- Classic numpy syntax (**only syntax allowed in exam**)
- [Einsum](https://rockt.ai/2018/04/30/einsum) (index-centric ops in NumPy/Torch)
- [Dumpy](https://dynomight.net/dumpy/) (loop-based NumPy)


## "Case studies"

- [AlexNet (2012) code](https://github.com/computerhistory/AlexNet-Source-Code): Python & custom binds to self-implemented CUDA kernels
- [karpathy/micrograd](https://github.com/karpathy/micrograd): Educational tensor abstraction in Python's stdlib

What happens in torch's backend when instantiating a tensor?

In [ ]:
import torch
x = torch.tensor([1,2,3])

[Torch source code](https://github.com/pytorch/pytorch):
1. `import torch` calls `torch/__init__.py`: brings functions into namespace (incl. C-bindings via `from torch._C import *`)
2. Create instance via `torch/_tensor.py` ([link](https://github.com/pytorch/pytorch/blob/31d12b3955363b1dd45d2938f9cca08436c43387/torch/_tensor.py#L102)) –> inherits from C base (`torch._C.TensorBase`) + Python with C-bindings

> **💡 Torch: complex bindings to fast lower-level code & clean interface via Python (e.g., dunders, see later lectures)**

## Compilation

Recap:
- Python interpreter: traverses code line by line **during runtime**
- Calling C bindings as-we-go: "eager mode"

Compilation/"lazy" mode: adaptively mapping **larger** codeblocks into single kernels 

### Example: [JAX](https://kidger.site/thoughts/torch2jax/)

JIT (just-in-time) compilation:

- Decorate function with `@jax.jit`
- First call: Python traces the ops
- After tracing: compile a kernel and run it

## Acceleration "without Python"
- [Mojo](https://www.modular.com/mojo)
  - Pro: everything written in one language (incl. compiler for any GPU vendor)
  - Con: less mature; less adoption
- [Julia](https://kidger.site/thoughts/jax-vs-julia/)
  - Pro: more expressive than Python, incl. GPU backend
  - Con: less adoption

# Example tasks

Goal: map some logic (e.g., math, slow Python code) to tensor ops

## Task A (make slow Python code fast)

In [ ]:
def slow(x: np.ndarray):
    d1, d2 = x.shape
    r = np.zeros((2, d1))
    for i in range(d1):
        for j in range(d2):
            for k in range(2):
                r[k, i] += (x[k, j] - x[i, j]) ** 2
    return r

Prepare testing:

In [ ]:
seed = 42
rng = np.random.default_rng(seed)
x = rng.random((3, 7))  # d1 must be ≥2
slow_result = slow(x)
slow_result.shape

### Solution strategies

> **⚠️ There are many ways to approach such tasks. Find the one that is suitable to \*you\*.**

#### Simplify problem

Examples:

- Focus on subset of ops
- Focus on computation (RHS of `=`) not where data is saved (LHS of `=`)

#### Loop visualizations
*See lecture recording.*

1. Draw each array as a 2D grid.
2. Place axis arrows labeled by indices (e.g., vertical arrow `i` for `x` since `x[i,j]`).

**Heuristics:**
- Two arrows on same input axis → broadcasting
- Arrow on input but no corresponding arrow on output → axis reduction
- Arrow repositioned → broadcasting (e.g., `v[None]-v` moves first axis of `v` to second dimension) or transposition
- Output arrow spans fewer units than the original axis (e.g., 2 < d1) → slicing

#### "Shape matching"
How to get from input shape (`d1 x d2`) to output shape (`2 x d1`)?

Transform `x` to `t1` and `t2`:

```
t1      (3d array): 1 x d1 x d2  | Add singleton dim
t2      (3d array): 2 x 1  x d2  | Slice first dim & add singleton dim
result  (2d array): 2 x d1       | Reduce over d2
```

#### Formalization

$$
r_{k,i} = \sum_{j=0}^{d_2-1} (x_{k,j} - x_{i,j})^2,
$$
where:
$x \in \mathbb{R}^{d1 \times d2}, \quad r \in \mathbb{R}^{2 \times d1}, \quad k \in \{0, 1\}, \quad i \in \{0, 1, \ldots, d_1-1\}$

### Solutions

In [ ]:
def slow(x):  # Copy function for reference
    d1, d2 = x.shape
    r = np.zeros((2, d1))
    for i in range(d1):
        for j in range(d2):
            for k in range(2):
                r[k, i] += (x[k, j] - x[i, j]) ** 2
    return r

In [ ]:
def fast(x):
    return ((x[:2, None] - x) ** 2).sum(2)  # Identical in torch

np.allclose(slow(x), fast(x))

In [ ]:
def inefficient(x):
    tmp = (x[:2] - x[:, None])
    tmp = ((tmp) ** 2).sum(2)
    result = tmp.transpose(1, 0)  # equivalent to tmp.T
    return result

np.allclose(slow(x), inefficient(x))

```
t1      (3d array): 1  x 2 x d2 
t2      (3d array): d1 x 1 x d2  | Slicing here would be more efficient
result  (2d array): d1 x 2       | reduced over d2 (before transposition)
```


## Task B (make slow Python code fast)

In [ ]:
def slow(x: torch.Tensor):
    d1, d2 = x.shape
    assert d1 > 1 and d2 > 1  # This may not be stated in exam
    r = torch.empty((d2, d1))  # `torch.zeros` may be more robust in practice if we accidentally don't overwrite all values
    for i in range(d1):
        for j in range(d2):
            r[j, i] = (i != j) * x[i, j] ** 2
    return r

In [ ]:
torch.manual_seed(42)
x = torch.rand(5, 7)
slow_result = slow(x)
slow_result.shape

Definition of $r$:
$$
r_{j, i} =
\begin{cases}
x_{i, j}^2 & \text{if } i \neq j \\
0 & \text{if } i = j
\end{cases}
$$

In [ ]:
# from pdb import set_trace
# Place `set_trace()` in code to debug


def fast(x):
    idx = torch.arange(min(x.shape))
    x[idx, idx] = 0  # in-place, not as in `slow`: whether this is helpful depends on context
    return (x ** 2).transpose(0, 1)


# torch.all(slow(x) == fast(x))
torch.allclose(slow(x), fast(x))  # account for numerical errors

## Attention

- After this lecture: know how to map formulas to torch/numpy
- Conceptual/ML part (what is attention?) **not** yet required / covered by **future** lecture
  - Lecture video: uses [this tool](https://poloclub.github.io/transformer-explainer/) to introduce **optional** context (e.g., for students already familiar with attention)

Future: merge some core numpy/torch lecture content with applications lectures, e.g., do **full** conceptual discussion of attention **before** discussing the respective "tensor mechanics"? Feel free to provide feedback whether this would be more interesting (closer to applications) or overwhelming (more complexity).

## Formalizing attention

Shapes:
- $B$: batch size
- $T$: sequence (of tokens)
- $N$: (attention) heads
- $D$: embed dim (for one head)

Input tensor (e.g., a ChatGPT user query embedded in a neural net feature space):
$$
X \in \mathbb{R}^{B \times T \times ND}
$$

Attain tensors below by multiplying $X$ with weight matrices & adjusting shape:

$$Q,K,V\in\mathbb{R}^{B\times N\times T\times D}

Define mask (will mask out parts of the attention matrix later):
$$
M\in\{0,-\infty\}^{B\times N\times T\times T}
$$

Attention matrix (two different notations; the latter notation only captures some dimensions):
$$
S=QK^\top, \qquad S_{ij}=\sum_{k=1}^D Q_{ik}K_{jk}
$$

Normalize attention matrix (divide by embed dim size) & apply mask:
$$
\tilde S=\frac{S+M}{\sqrt{D}}
$$

Apply numerically stable softmax:
- [Softmax function](https://en.wikipedia.org/wiki/Softmax_function): create probability distribution
- Shift-invariance → shift to smaller values to avoid numerical problems (row-wise)
- $S+M$: if $S$ is finite & $M$ is $-\infty$ → zero probability mass under softmax
$$
P_{ij}=\frac{e^{\tilde S_{ij}-m_i}}{\sum_{j'} e^{\tilde S_{ij'}-m_i}}, \qquad m_i=\max_j \tilde S_{ij}
$$

Matmul: attention matrix & $V$ (two different notations):
$$
O=PV, \qquad O_{i:}=\sum_j P_{ij}V_{j:}
$$

## Coding simplified attention

- Set $B=1$ and $N=1$
- Assume $Q$, $K$, and $V$ are given

In [ ]:
import math

def attention_core(Q, K, V, mask):
    """
    Q,K,V: (T, D)
    mask: (T, T) with 0 or -inf
    returns O: (T, D)
    """
    T, D = Q.shape
    S = (Q @ K.T) * (1.0 / math.sqrt(D))   # (T, T)
    S = S + mask
    # manual softmax; F.softmax(S, dim=-1) from `from torch.nn import functional as F` is equivalent
    m = S.max(dim=-1, keepdim=True).values
    P = (S - m).exp()
    P = P / P.sum(dim=-1, keepdim=True)
    O = P @ V                               # (T, D)
    return O


## Coding full attention

In [ ]:
import math

def attention_full(x, Wq, Wk, Wv, M, N):
    """
    x:  (B, T, N*D)
    Wq,Wk,Wv: (N*D, N*D)
    M:  (B, N, T, T) with 0 or -inf
    returns y: (B, T, N*D)
    """
    B, T, C = x.shape  # C = N*D
    assert C % N == 0
    D = C // N

    # "neural net projection"
    q = x @ Wq                      # (B,T,N*D)
    k = x @ Wk                      # (B,T,N*D)
    v = x @ Wv                      # (B,T,N*D)

    q = q.view(B, T, N, D).transpose(1, 2).contiguous()  # (B,N,T,D)
    k = k.view(B, T, N, D).transpose(1, 2).contiguous()  # (B,N,T,D)
    v = v.view(B, T, N, D).transpose(1, 2).contiguous()  # (B,N,T,D)

    S = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(D))  # (B,N,T,T)
    S = S + M

    # manual softmax; F.softmax(S, dim=-1) is equivalent
    m = S.max(dim=-1, keepdim=True).values
    P = (S - m).exp()
    P = P / P.sum(dim=-1, keepdim=True)

    y = P @ v                                            # (B,N,T,D)
    y = y.transpose(1, 2).contiguous().view(B, T, C)     # (B,T,N*D)

    return y


"In the wild":
[karpathy/nanoGPT](https://github.com/karpathy/nanoGPT/blob/93a43d9a5c22450bbf06e78da2cb6eeef084b717/model.py#L29)

Follow-up: [karpathy/nanochat](karpathy/nanochat)